In [68]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(stringr))
library(ggrepel)
library(reshape2)
library(ggsci)
library(rstatix)
library(ggpubr)
library(hise)

In [69]:
source("../..//helper_function//helper_function_IHA.r")

# Get DEG Number

In [70]:
DEG_res<-read.csv("../02_DEG/Deseq2_Result_Y1D0.csv")

In [71]:
DEG_res_signi<-DEG_res %>% filter(padj<0.05,abs(log2FoldChange)>0.1)

In [72]:
DEG_counts<-as.data.frame(table(DEG_res_signi$contrast,DEG_res_signi$celltype))

In [73]:
colnames(DEG_counts)<-c("factor","AIFI_L3","DEG_Counts")

# Get Freq

In [74]:
BRI_Freq_Data<-read.csv("../../Dataset/diha_AIFI_L3_frequencies_2024-05-05.csv")

In [75]:
BRI_Freq_Data<-BRI_Freq_Data %>% group_by(sample.sampleKitGuid) %>%
  dplyr::mutate(percentage = (AIFI_L3_count / sum(AIFI_L3_count)) * 100) %>% 
  dplyr::mutate(AIFI_L3_clr = clr_transform(percentage)) %>% ungroup()

In [76]:
BRI_Freq_Data_subset<-BRI_Freq_Data %>% filter(sample.visitName=="Flu Year 1 Day 0")

### AgeGroup

In [77]:
#compute freq changes
freq_changes<-BRI_Freq_Data_subset %>%
  select(percentage, AIFI_L3_clr,AIFI_L3, cohort.cohortGuid, subject.biologicalSex, subject.cmv) %>%
  group_by(AIFI_L3, cohort.cohortGuid) %>%
  summarise(mean_AIFI_L3_clr = median(AIFI_L3_clr, na.rm = TRUE)) %>%
  spread(cohort.cohortGuid, mean_AIFI_L3_clr) %>%
  mutate(log2fc = log2(`BR2` / `BR1`)) %>%
  mutate(delta_change = `BR2` - `BR1`) 
#compute freq changes p values

freq_changes_STAT<-BRI_Freq_Data_subset %>%
  select(AIFI_L3_clr,AIFI_L3, cohort.cohortGuid,subject.biologicalSex,subject.cmv) %>%
  group_by(AIFI_L3)  %>%
  wilcox_test(AIFI_L3_clr ~ cohort.cohortGuid) %>%
  adjust_pvalue(method='BH') %>%
  add_significance() %>% as.data.frame()

df<-left_join(freq_changes_STAT,freq_changes)

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.
Warning message:
“There were 4 warnings in `mutate()`.
The first warning was:
ℹ In argument: `log2fc = log2(BR2/BR1)`.
ℹ In group 3: `AIFI_L3 = "Adaptive NK cell"`.
Caused by warning:
! NaNs produced
ℹ Run `dplyr::last_dplyr_warnings()` to see the 3 remaining warnings.”
Joining with `by = join_by(AIFI_L3)`


In [78]:
df<-left_join(df,DEG_counts %>% filter(factor=="cohort.cohortGuid"))

Joining with `by = join_by(AIFI_L3)`


In [79]:
df<-df%>%
  mutate(adjP_degs_combined_value = if_else(DEG_Counts != 0,
                                            -log10(p.adj)  * DEG_Counts ,
                                            -log10(p.adj)  ))

In [80]:
write.csv(df,'FreqChanges_and_DEG.csv')